In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from matplotlib.colors import ListedColormap
from sklearn.tree import DecisionTreeClassifier

## Load training/test data

In [2]:
f = open('parkinsonsTrainStatML.dt')
content = f.read()
lines = np.array([np.array(list(map(float, line.split(' ')))) for line in content.split('\n')[:-1]])
ft = open('parkinsonsTestStatML.dt')
contentt = ft.read()
linest = np.array([np.array(list(map(float, line.split(' ')))) for line in contentt.split('\n')[:-1]])

## Scale data to zero mean, unit variance

In [3]:
means = np.array([np.mean(lines[:,i]) for i in range(len(lines[1,:])-1)])
stds = np.array([np.std(lines[:,i]) for i in range(len(lines[1,:])-1)])

x_normed = (lines[:,:-1] - lines[:,:-1].mean(axis=0)) / lines[:,:-1].std(axis=0)
x_normedt = (linest[:,:-1] - means) / stds
y_train = lines[:,-1]
y_test = linest[:,-1]
#np.savetxt('normed.txt', normed)
#np.savetxt('normedt.txt', normedt)

Helper function, for FP, FN, TP, TN

In [4]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1.0:
           TP += 1
        if y_hat[i]==1.0 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0.0:
           TN += 1
        if y_hat[i]==0.0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [13]:
models = [
    RandomForestClassifier(max_depth=None, min_samples_split=3, n_estimators=100),
    DecisionTreeClassifier(max_depth=None)
]


cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses
RANDOM_SEED = 13  # fix the seed on each iteration
plot_idx = 1

for pair in ([0, 1], [0, 2], [10, 11]):
    for model in models:
        # We only take the two corresponding features
        X = x_normedt[0:20, pair]
        y = y_test[0:20]

        # Shuffle
        idx = np.arange(X.shape[0])
        np.random.seed(0)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]
        # Train
        model.fit(X, y)
        
        scores = model.score(X, y)
        # Create a title for each column and the console by using str() and
        # slicing away useless parts of the string
        model_title = str(type(model)).split(
            ".")[-1][:-2][:-len("Classifier")]

        model_details = model_title
        if hasattr(model, "estimators_"):
            model_details += " with {} estimators".format(
                len(model.estimators_))
        print(model_details + " with features", pair,
              "has a score of", scores)

        plt.subplot(3, 2, plot_idx)
        axs[0].set_title('subplot 1')
        if plot_idx <= len(models):
            # Add a title at the top of each column
            plt.title(model_title, fontsize=9)
            plt.subtitle(f'Features: {pair}', fontsize=9)

        # Now plot the decision boundary using a fine mesh as input to a
        # filled contour plot
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                             np.arange(y_min, y_max, plot_step))

        # Plot either a single DecisionTreeClassifier or alpha blend the
        # decision surfaces of the ensemble of classifiers
        if isinstance(model, DecisionTreeClassifier):
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            cs = plt.contourf(xx, yy, Z, cmap=cmap)
        else:
            # Choose alpha blend level with respect to the number
            # of estimators
            # that are in use (noting that AdaBoost can use fewer estimators
            # than its maximum if it achieves a good enough fit early on)
            estimator_alpha = 1.0 / len(model.estimators_)
            for tree in model.estimators_:
                Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

        # Build a coarser grid to plot a set of ensemble classifications
        # to show how these are different to what we see in the decision
        # surfaces. These points are regularly space and do not have a
        # black outline
        xx_coarser, yy_coarser = np.meshgrid(
            np.arange(x_min, x_max, plot_step_coarser),
            np.arange(y_min, y_max, plot_step_coarser))
        Z_points_coarser = model.predict(np.c_[xx_coarser.ravel(),
                                         yy_coarser.ravel()]
                                         ).reshape(xx_coarser.shape)
        cs_points = plt.scatter(xx_coarser, yy_coarser, s=15,
                                c=Z_points_coarser, cmap=cmap,
                                edgecolors="none")

        # Plot the training points, these are clustered together and have a
        # black outline
        plt.scatter(X[:, 0], X[:, 1], c=y,
                    cmap=ListedColormap(['r', 'y', 'b']),
                    edgecolor='k', s=20)
        plot_idx += 1  # move on to the next plot in sequence

plt.suptitle("Classifiers on feature subsets of the Iris dataset", fontsize=12)
plt.axis("tight")
plt.tight_layout(h_pad=0.2, w_pad=0.2, pad=2.5)        
plt.savefig('RF.png')

RandomForest with 100 estimators with features [0, 1] has a score of 1.0


TypeError: 'Text' object is not callable

## Hyperparameter grid search with 5 fold cross validation

In [6]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}
]

param_grid_RF = [
   {'n_estimators': [100, 200], 'max_depth': [None, 10], 'min_samples_split': [2,3]}
]

In [7]:
scores = ['accuracy', 'precision', 'recall']
best_parameters_RF = {}
predictions = {}
for score in scores:
    print(f"# Tuning hyper-parameters for {score}")
    clf = GridSearchCV(
        RandomForestClassifier(), param_grid_RF, scoring=score, cv=5
    )
    clf.fit(x_normed, y_train)
    best_parameters_RF[score] = clf.best_params_
    y_true, predictions[score] = y_test, clf.predict(x_normedt)
    TP, FP, TN, FN = perf_measure(y_true, predictions[score])
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')
    print(classification_report(y_true, predictions[score]))
print(best_parameters_RF)

# Tuning hyper-parameters for accuracy
TP: 71, FP: 9, TN: 13, FN: 4
              precision    recall  f1-score   support

         0.0       0.76      0.59      0.67        22
         1.0       0.89      0.95      0.92        75

    accuracy                           0.87        97
   macro avg       0.83      0.77      0.79        97
weighted avg       0.86      0.87      0.86        97

# Tuning hyper-parameters for precision
TP: 69, FP: 7, TN: 15, FN: 6
              precision    recall  f1-score   support

         0.0       0.71      0.68      0.70        22
         1.0       0.91      0.92      0.91        75

    accuracy                           0.87        97
   macro avg       0.81      0.80      0.81        97
weighted avg       0.86      0.87      0.86        97

# Tuning hyper-parameters for recall
TP: 70, FP: 8, TN: 14, FN: 5
              precision    recall  f1-score   support

         0.0       0.74      0.64      0.68        22
         1.0       0.90      0.93 

In [8]:
scores = ['accuracy', 'precision', 'recall']
best_parameters = {}
predictions = {}
for score in scores:
    print(f"# Tuning hyper-parameters for {score}")
    clf = GridSearchCV(
        SVC(), param_grid, scoring=score, cv=5
    )
    clf.fit(x_normed, y_train)
    best_parameters[score] = clf.best_params_
    y_true, predictions[score] = y_test, clf.predict(x_normedt)
    TP, FP, TN, FN = perf_measure(y_true, predictions[score])
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')
    print(classification_report(y_true, predictions[score]))
print(best_parameters)

# Tuning hyper-parameters for accuracy
TP: 74, FP: 13, TN: 9, FN: 1
              precision    recall  f1-score   support

         0.0       0.90      0.41      0.56        22
         1.0       0.85      0.99      0.91        75

    accuracy                           0.86        97
   macro avg       0.88      0.70      0.74        97
weighted avg       0.86      0.86      0.83        97

# Tuning hyper-parameters for precision
TP: 74, FP: 13, TN: 9, FN: 1
              precision    recall  f1-score   support

         0.0       0.90      0.41      0.56        22
         1.0       0.85      0.99      0.91        75

    accuracy                           0.86        97
   macro avg       0.88      0.70      0.74        97
weighted avg       0.86      0.86      0.83        97

# Tuning hyper-parameters for recall
TP: 75, FP: 22, TN: 0, FN: 0
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        22
         1.0       0.77      1.00 

In [9]:
best_parameters['accuracy']

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}